In [1]:
import numpy as np

In [2]:
from scipy.stats import norm

class OptionHedgeEnv():

  def __init__(self, r, T, vol, dt=1, x0=100, k=0.01):
    '''
    _state: np.array([hedge position, stock_price, time to maturity])
    _stock_price: np.array of stock price with length T+1
    _time_to_maturity
    _strike_price
    _episode_ended: if this episode is ended
    '''
    self._stock_price = self._generate_stockprice(r, T, vol, dt, x0)
    self._time_to_mature = T
    self._strike_price = x0
    self._episode_ended = False
    self._k = k
    self._vol = vol
    self._r = r
    self._action = self._generate_action()
    self._option_value = self._generate_option_value()
    self._state = np.array([0, 100, T], dtype = np.float32)

  def _generate_action(self):
    T = self._time_to_mature
    d1 = np.log(self._stock_price[:-1]/self._strike_price) + (self._r+self._vol**2/2)/252*np.array(list(range(T,0,-1)))
    d1 = d1/(self._vol * np.sqrt(np.array(list(range(T,0,-1)))/252))
    return norm.cdf(d1)

  def _generate_stockprice(self, r, T, vol, dt=1, x0=100):
    '''
    r: annualized return
    T: option time
    vol: annualized volatility
    dt: observing time interval
    x0: initial price

    return: np.array with shape (T,)
    '''
    mu = (1+r)**(1/252)-1
    sigma = vol/np.sqrt(252)
    x = np.exp((mu - sigma ** 2 / 2) * dt + sigma * np.random.normal(0, np.sqrt(dt), size=round(T/dt)))
    x = x0 * x.cumprod(axis=0)
    x = np.insert(x, 0, x0)
    return x

  def _generate_option_value(self):
    T = self._time_to_mature
    d1 = np.log(self._stock_price[:-1]/self._strike_price) + (self._r+self._vol**2/2)/252*np.array(list(range(T,0,-1)))
    d1 = d1/(self._vol * np.sqrt(np.array(list(range(T,0,-1)))/252))
    d2 = d1 - self._vol * np.sqrt(np.array(list(range(T,0, -1)))/252)
    C = (norm.cdf(d1) - np.exp(-np.array(list(range(T,0,-1)))/252*self._r)*norm.cdf(d2))*self._strike_price
    return np.append(C, self._stock_price[-1]-self._strike_price)

  def _calculate_cashflow(self, new_state, old_state):
    cashflow = - self._k*np.abs(old_state[1] * (old_state[0]-new_state[0]))
    return cashflow
  
  def _value_change(self, new_state, old_state):
    value_diff = new_state[0] * (new_state[1]-old_state[1]) - (self._option_value[-int(new_state[2])-1]-self._option_value[-int(old_state[2])-1])
    return value_diff

  def _reset(self):
    self._state = np.array([0, 100, self._time_to_mature], dtype = np.float32)
    self._stock_price = self._generate_stockprice(self._r, self._time_to_mature, self._k)
    self._episode_ended = False
    self._action = self._generate_action()
    self._option_value = self._generate_option_value()

  def _step(self, action):

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    old_state = self._state
    new_state = np.array([0, 0, 0], dtype=np.float32)
    new_state[0] = action
    new_state[2] = old_state[2] - 1
    time_to_mature = int(new_state[2])
    new_state[1] = self._stock_price[-time_to_mature-1]
    # Make sure episodes don't go on forever.
    if time_to_mature == 0:
      self._episode_ended = True

    if self._episode_ended or time_to_mature == self._time_to_mature-1:
      reward = self._calculate_cashflow(new_state, old_state)
      reward += self._value_change(new_state, old_state)
      reward -= self._k * np.abs(new_state[0] * new_state[1])
      self._state = new_state
      return reward
    else:
      reward = self._calculate_cashflow(new_state, old_state)
      reward += self._value_change(new_state, old_state)
      self._state = new_state
      return reward

  def hedge(self, rounds):
    rewards = []
    for i in range(rounds):
      self._reset()
      k = 0
      total_re = 0      
      while not self._episode_ended:
        action = self._action[k]
        reward = self._step(action)
        total_re += reward
        k += 1
      rewards.append(total_re)
    return rewards
        

In [3]:
env = OptionHedgeEnv(0.05, 5, 0.2)

In [10]:
hedging_cost = env.hedge(10000)

In [11]:
np.mean(hedging_cost) - env._option_value[0]

-1.7967107337564365

In [12]:
np.std(hedging_cost)

0.1671511116869143

In [7]:
0.1307 - env._option_value[0]

-1.5581200890873839

In [8]:
1.558/1.796

0.8674832962138085

In [13]:
0.13/0.167

0.7784431137724551